## Wildcards and typos

For a predefined (small) [vocabulary](../datasets/small-dict.txt), which you can download or even hardcode in the solution, you should be able to solve 2 types of problems:
1. Retrieve all words, matching a given **wildcard**.
2. Retrieve the best suggestion for **typo** fix.

To know, how to do this, you can adress the book, lecture slides and homework statement.

### input

You will be given a file `input.txt` with 2 lines. The first will have the type of action (`wildcard` or `typo`), the second will store input data. Like this:
```
typo
anemal
```
or 
```
wildcard
a*al
```

### output

Print result into a `output.txt` file. 
- _Wildcard_ can return multiple results, so print them in alphabetic order, one per line. Like this:

```
analitical
animal
annual
```

- _Typo_ fix should return the single best match (there will be always ot least one good match).

### prohibited things
- Regular expressions (`import re` in particular).
- testing wildcards with `part` **in** `word`.

In [4]:
import requests
from collections import Counter, defaultdict

#words = requests.get('https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/small-dict.txt').text.split()
words = ['caterpillar', 'alligator', 'crocodile', 'groundhog', 'elephant', 'cheetah', 'butterfly', 'anaconda', 'wolverine', 'wolf', 'firefox', 'badger', 'buffalo', 'cassowary', 'chinchilla', 'chimpanzee', 'flamingo', 'grasshopper', 'hummingbird', 'penguin', 'bluebird']
class SpellingCorrector:
    def __init__(self, words, 
                 edit1coeff=1e-3, edit2coeff=1e-6, 
                 absent_word_p=1e-4, absent_pair_p=1e-8, 
                 elim_coeff=2, cand_elim_coeff=None):
        self.words = words
        self.e1k = edit1coeff
        self.e2k = edit2coeff
        self.awp = absent_word_p
        self.app = absent_pair_p
        self.EK = elim_coeff
        self.CEK = cand_elim_coeff
        self.N = len(words)
        self.probs = dict()

    def known(self, words):
        # The subset of `words` that appear in the dictionary of WORDS.
        return set(w for w in words if w in self.words)

    def edits1(self, word):
        # All edits that are one edit away from `word`.
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        # All edits that are two edits away from `word`.
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

    def candidates(self, word):
        # Generate possible spelling corrections for `word` 
        # with their probabilities.
        result = []
        if len(self.known([word])) > 0:
            result += [(word, 1)]
        
        t = self.known(self.edits1(word))
        if len(t) > 0:
            result += [(ti, self.e1k) for ti in t]
        
        t = self.known(self.edits2(word))
        if len(t) > 0:
            result += [(ti, self.e2k) for ti in t]
        
        if len(result) == 0:
            result = [(word, 1)]
        elif self.CEK is not None:
            result = sorted(result, key=self.word_p, reverse=True)[:self.CEK]
        return result

    def word_p(self, word):
        # return `word` probability
        return 1 / self.N if word in self.words else self.awp

    def pair_p(self, pair):
        # return `pair` probability
        return PAIRS[pair] / self.M if pair in self.pairs else self.app

    def P(self, query):
        # Probability of `query`.
        result = 1
        
        # check if the part of the query is cached
        tmp = self.unpack(query[:-1])
        if len(query) > 1 and tmp in self.probs:
            w, p = query[-1]
            wl, _ = query[-2]
            r = self.probs[tmp]
            return r * self.pair_p(wl + ' ' + w) * self.word_p(w) * p
        
        wl = ""
        for w, p in query:
            if wl == "":
                result *= self.word_p(w) * p
            else:
                result *= self.pair_p(wl + ' ' + w) * self.word_p(w) * p
            wl = w
        return result

    def unpack(self, arr):
        # convert a list of word-prob tuples to a string
        return " ".join([w for (w, _) in arr])

    def eliminate(self, arr):
        # return `n` most probable distinct sequences
        arr = sorted(arr, key=self.P, reverse=True)
        
        i = 0
        result = []
        tmp = set()
        while len(result) < self.EK and i < len(arr):
            t = self.unpack(arr[i])
            if t not in tmp:
                result += [arr[i]]
                tmp.add(t)
            i += 1
        
        # cache the probabilities
        probs = dict()
        for x in result:
            probs[self.unpack(x)] = self.P(x)
        self.probs = probs
        return result

    def print_arr(self, arr):
        # debug print most probable variants
        print("-----")
        for t in arr:
            print(" ".join([w for w, _ in t]), ":", self.P(t))
        print("-----")

    def correct(self, query, debug=False):
        # Most probable spelling correction for query.
        tmp = []
        result = []
        probs = dict()
        for w in query.strip().lower().split():
            tmp = result
            if len(tmp) == 0:
                result = [[c] for c in self.candidates(w)]
            else:
                result = []
                for t in tmp:
                    result.extend([t + [c] for c in self.candidates(w)])
            result = self.eliminate(result)
        if debug:
            self.print_arr(result)
        return self.unpack(max(result, key=self.P))


with open('input.txt', 'r') as f:
    action = f.readline().strip()
    data = f.readline().strip()
    sp = SpellingCorrector(words=words)
    if 'typo' == action:
        with open("output.txt", "w") as fo:
            fo.write(sp.correct(data))
    elif 'wildcard' == action:
        index = defaultdict(list)
        for w in words:
            word = "^{}$".format(w)
            for i in range(len(word)):
                gram = word[i:i+1]
                index[gram].append(w)
            for i in range(len(word) - 1):
                gram = word[i:i+2]
                index[gram].append(w)
            for i in range(len(word) - 2):
                gram = word[i:i+3]
                index[gram].append(w)
        
        data = "^{}$".format(data)
        patterns = data.split("*")
        candidates = set(words.copy())
        for pattern in patterns:
            for i in range(len(pattern) - 1):
                temp = index.get(pattern[i:i+2], [])
                candidates = candidates.intersection(temp)
            for i in range(len(pattern) - 2):
                temp = index.get(pattern[i:i+3], [])
                candidates = candidates.intersection(temp)
            for i in range(len(pattern)):
                temp = index.get(pattern[i:i+1], [])
                candidates = candidates.intersection(temp)
        candidates = sorted(candidates)
        with open("output.txt", "w") as fo:
            for c in candidates:
                fo.write(c + "\n")
        raise Exception(data)

Exception: ^a*da$

In [7]:
print(sorted(words))

['alligator', 'anaconda', 'badger', 'bluebird', 'buffalo', 'butterfly', 'cassowary', 'caterpillar', 'cheetah', 'chimpanzee', 'chinchilla', 'crocodile', 'elephant', 'firefox', 'flamingo', 'grasshopper', 'groundhog', 'hummingbird', 'penguin', 'wolf', 'wolverine']


In [21]:
data = "^*ing*$"



In [28]:
data = "^*l*r$"

In [30]:
data = "^ch*e*$"

In [31]:
index = defaultdict(list)
for w in words:
    word = "^{}$".format(w)
    for i in range(len(word)):
        gram = word[i:i+1]
        index[gram].append(w)
    for i in range(len(word) - 1):
        gram = word[i:i+2]
        index[gram].append(w)
    for i in range(len(word) - 2):
        gram = word[i:i+3]
        index[gram].append(w)
        
data = "^{}$".format(data)
patterns = data.split("*")
candidates = set(words.copy())
for pattern in patterns:
    for i in range(len(pattern) - 1):
        temp = index.get(pattern[i:i+2], [])
        candidates = candidates.intersection(temp)
    for i in range(len(pattern) - 2):
        temp = index.get(pattern[i:i+3], [])
        candidates = candidates.intersection(temp)
    for i in range(len(pattern)):
        temp = index.get(pattern[i:i+1], [])
        candidates = candidates.intersection(temp)
candidates = sorted(candidates)
print(candidates)

['cheetah', 'chimpanzee']


In [26]:
patterns

['^', 'l', 'r$']

In [32]:
print(words)

['caterpillar', 'alligator', 'crocodile', 'groundhog', 'elephant', 'cheetah', 'butterfly', 'anaconda', 'wolverine', 'wolf', 'firefox', 'badger', 'buffalo', 'cassowary', 'chinchilla', 'chimpanzee', 'flamingo', 'grasshopper', 'hummingbird', 'penguin', 'bluebird']
